In [1]:
# Load Depedencies

import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.models as models
import torch.optim as optim
import os
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torchattacks
import matplotlib.pyplot as plt
from torchmetrics.functional.image import peak_signal_noise_ratio, structural_similarity_index_measure
from trades import trades_loss

In [2]:
model_name = "VGG16"
version = "v1"
training_name = "TRADES"
num_classes = 7
batch_size = 16
learning_rate = 0.0001
lr_factor = 0.1
lr_threshold = 5
epochs = 100
height = 224
save_freq = 5

epsilon = 8.0/255
alpha = 2.0/255
steps = 4
beta = 0.1

In [3]:
# Setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Graph writer initialize
writer = SummaryWriter("runs/trashbox/" + f'{training_name}--{model_name}.{version}')

In [4]:
preprocessing = transforms.Compose([
    transforms.RandomResizedCrop((height, height)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
test_preprocessing = transforms.Compose([
    transforms.RandomResizedCrop((height, height)),
    transforms.ToTensor(),
])

In [5]:
trash_train_dataset = torchvision.datasets.ImageFolder('dataset/trashbox/train', transform=preprocessing)
trash_train_loader = torch.utils.data.DataLoader(dataset=trash_train_dataset, shuffle=True, batch_size=batch_size)
trash_val_dataset = torchvision.datasets.ImageFolder('dataset/trashbox/val', transform=test_preprocessing)
trash_val_loader = torch.utils.data.DataLoader(dataset=trash_val_dataset, shuffle=True, batch_size=batch_size)

In [6]:
model = models.vgg16()
model.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=num_classes, bias=True)
  )

model = model.to(device)

In [7]:
def train(model, device, train_loader, optimizer, epoch): 
    print(f'Train epoch no. [{epoch}]')
    train_loss = 0
    adv_correct = 0
    nat_correct = 0
    total_ssim = 0
    total_psnr = 0
    total = 0
    model.train()
    iterator = tqdm(train_loader, ncols=0, leave=False)
    for i, (inputs, targets)in enumerate(iterator):
        inputs, targets = inputs.to(device),targets.to(device)

        optimizer.zero_grad()
        nat_output = model(inputs)

        
        loss  = trades_loss(
            model=model, 
            x_natural=inputs,
            y=targets,
            optimizer=optimizer,
            step_size=alpha,
            epsilon=epsilon,
            perturb_steps=steps,
            beta=beta,
            distance='l_inf'
        )
      

        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        total += targets.size(0)

        _, nat_predict = nat_output.max(1)
        nat_correct += nat_predict.eq(targets).sum().item()
        
    total_train_loss = train_loss / len(train_loader)

    print('\nTotal adv train accuarcy:', 100. * adv_correct / total)
    print('Validation loss:', total_train_loss)

    writer.add_scalar('Train loss: ' + model_name, total_train_loss, epoch)
    writer.add_scalar('Natural Train Accuracy: ' + model_name, 100. * nat_correct / total, epoch)
    
    return train_loss

In [8]:
def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    nat_correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            nat_output = model(data)
            test_loss += F.cross_entropy(nat_output, target, size_average=False).item()
            pred = nat_output.max(1, keepdim=True)[1]
            nat_correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, nat_correct, len(test_loader.dataset),
        100. * nat_correct / len(test_loader.dataset)))
    test_accuracy = nat_correct / len(test_loader.dataset)

    return test_loss, test_accuracy

In [9]:
def adjust_learning_rate(optimizer, epoch):
    """decrease the learning rate"""
    lr = learning_rate
    if epoch >= 75:
        lr = learning_rate * 0.1
    if epoch >= 90:
        lr = learning_rate * 0.01
    if epoch >= 100:
        lr = learning_rate * 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [10]:
def main():
    # init model, ResNet18() can be also used here for training
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)
    best_acc = float(0)
    for epoch in range(1, epochs + 1):
        # adjust learning rate for SGD
        adjust_learning_rate(optimizer, epoch)

        # adversarial training
        train(model, device, trash_train_loader, optimizer, epoch)

        # evaluation on natural examples
        print('================================================================')
        test_loss, test_acc = eval_test(model, device, trash_val_loader)
        print('================================================================')

        # Graph
        writer.add_scalar("Natural test loss: " + model_name, test_loss, epoch)
        writer.add_scalar("Natural test accuracy: " + model_name, test_acc, epoch)
        
        # Save checkpoint
        state = {
            'epoch' : epoch,
            'net': model.state_dict(),
            'optim' : optimizer.state_dict()
        }

        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        if epoch % save_freq == 0:
            torch.save(state, './checkpoint/' + f'{training_name}--{model_name}.{version}.pth')
        if test_acc > best_acc:
            print("Model saved @ acc: ", test_acc)
            torch.save(state, './trained_model/' + f'best_{training_name}_{model_name}_{version}_epoch{epoch}.pth')


if __name__ == '__main__':
    main()

Train epoch no. [1]


  0% 0/893 [00:00<?, ?it/s]c:\Users\Acer\.conda\envs\thesisaimodel\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
  0% 1/893 [00:09<2:18:56,  9.35s/it]c:\Users\Acer\.conda\envs\thesisaimodel\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB. GPU  (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,
c:\Users\Acer\.conda\envs\thesisaimodel\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB. GPU  (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,


OutOfMemoryError: CUDA out of memory. Tried to allocate 98.00 MiB. GPU 